<a href="https://colab.research.google.com/github/kimheeseo/2022_AI_SW_Programmingstudy/blob/main/practice_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# libraries
import numpy as np
import scipy as sp
import pandas as pd

In [3]:
# read data
df = pd.read_csv('data07_diabetes.csv')
X = df.iloc[:,:-1]
y = df['Y']

In [4]:
df.head()

,age,sex,bmi,map,tc,ldl,hdl,tch,ltg,glu,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135


# **boston data set**

In [5]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.75,random_state=0)
#testset을 의도적으로 크게 설정하고, trainset을 작게하여, noise값을 크게 만들었다.

# **forward feature selection**

In [6]:
np.random.seed(0)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score #각각의 모델을 평가하기 위해서, cross validation 기법 활용

In [8]:
#따로 구현한 방법 : forward selection
vn = list(xtrain.columns) #variable name : vn ex) age, sex, bmi, map, fc, ...... 
f_sel= [] 
score = [] #score : 모델의 성능이 얼마인지를 나타내는 지표값.
for i in range(xtrain.shape[1]): #모든 column에 대해서 진행
    s = np.zeros(len(vn))
    for j in range(len(vn)):
        v = f_sel.copy()#f_sel(현재 selected된 값)을 copy한다.
        v.append(vn[j])
        x = xtrain[v]
        f = LinearRegression()
        cv_score = cross_val_score(f,x,ytrain,cv=5) #교차검증을 5개로 나눠서 진행
        s[j] = cv_score.mean()#j번째 score
    v = vn[s.argmax()] #가장 큰 값을 argmax로 선택한 후, v에 입력.
    f_sel.append(v)
    vn.remove(v)
    score.append(s.max())#제일 좋았던 scroe를 출력한다.
    print("%02d Selected:"%i,f_sel)
    print("%02d Score   :"%i,np.round(10000*np.array(score))/10000)
    #결과값을 보면 10번 반복되는 것을 확인할 수 있다.
    #00 selected : 초기에는 여러 변수 중 bmi를 test를 했다. 가장 좋은 결과값을 갖기 때문에.
    #01 selected : ltg 사용, 02 selected : hdl ......
    #bmi에서 under fitting, age에서는 over fitting이 발생한다.
    #09 score 중에서 가장 값이 큰, map : 0.5657이 가장 좋은 값을 갖는다. 5개를 넣었을 때, 가장 좋은 성능을 보인다.

00 Selected: ['bmi']
00 Score   : [0.3935]
01 Selected: ['bmi', 'ltg']
01 Score   : [0.3935 0.544 ]
02 Selected: ['bmi', 'ltg', 'hdl']
02 Score   : [0.3935 0.544  0.5556]
03 Selected: ['bmi', 'ltg', 'hdl', 'sex']
03 Score   : [0.3935 0.544  0.5556 0.5627]
04 Selected: ['bmi', 'ltg', 'hdl', 'sex', 'map']
04 Score   : [0.3935 0.544  0.5556 0.5627 0.5657]
05 Selected: ['bmi', 'ltg', 'hdl', 'sex', 'map', 'tch']
05 Score   : [0.3935 0.544  0.5556 0.5627 0.5657 0.5587]
06 Selected: ['bmi', 'ltg', 'hdl', 'sex', 'map', 'tch', 'tc']
06 Score   : [0.3935 0.544  0.5556 0.5627 0.5657 0.5587 0.5475]
07 Selected: ['bmi', 'ltg', 'hdl', 'sex', 'map', 'tch', 'tc', 'ldl']
07 Score   : [0.3935 0.544  0.5556 0.5627 0.5657 0.5587 0.5475 0.5342]
08 Selected: ['bmi', 'ltg', 'hdl', 'sex', 'map', 'tch', 'tc', 'ldl', 'glu']
08 Score   : [0.3935 0.544  0.5556 0.5627 0.5657 0.5587 0.5475 0.5342 0.5208]
09 Selected: ['bmi', 'ltg', 'hdl', 'sex', 'map', 'tch', 'tc', 'ldl', 'glu', 'age']
09 Score   : [0.3935 0.544  0

# **finally selected features**

In [9]:
f_sel_final = f_sel[:(np.array(score).argmax()+1)]

In [11]:
f_sel_final

['bmi', 'ltg', 'hdl', 'sex', 'map']

# **test on the test set**

In [12]:
from sklearn.linear_model import LinearRegression
f = LinearRegression()
f.fit(xtrain[f_sel_final],ytrain)
f.score(xtest[f_sel_final],ytest)

0.43841451014788213

In [13]:
# comparison with the full model
f.fit(xtrain,ytrain)
f.score(xtest,ytest)
#위의 값보다 약간이나마 성능이 안좋다는 것을 확인할 수 있다.

0.43510681849311494

참고사이트 : https://scikit-learn.org/stable/modules/feature_selection.html
Sequnetial Feature Selection 확인